In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import torch
import torchrl
import matplotlib.pyplot as plt
from tensordict import TensorDict
from flipper_training.environment.env import Env
from flipper_training.configs import *
from flipper_training.rl_objectives import *
from flipper_training.utils.heightmap_generators import *

/Users/davidkorcak/.venv/lib/python3.12/site-packages/networkx/release.py:138: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  date_info = datetime.datetime.utcfromtimestamp(int(os.environ.get('SOURCE_DATE_EPOCH', time.time())))


In [4]:
num_robots = 1

In [5]:
from flipper_training.utils.environment import make_x_y_grids, generate_heightmaps

In [6]:
# Heightmap setup - use torch's XY indexing !!!!!
grid_res = 0.05  # 5cm per grid cell
max_coord = 6.4  # meters
heighmap_gen = MultiGaussianHeightmapGenerator(
    min_gaussians=400,
    max_gaussians=600,
    min_height_fraction=0.03,
    max_height_fraction=0.12,
    min_std_fraction=0.03,
    max_std_fraction=0.08,
    min_sigma_ratio=0.6,
)
x_grid, y_grid = make_x_y_grids(max_coord, grid_res, num_robots)
z_grid, suit_mask = generate_heightmaps(x_grid, y_grid, heighmap_gen)

In [7]:
from flipper_training.vis.static_vis import plot_heightmap_3d

plot_heightmap_3d(x_grid[0], y_grid[0], z_grid[0])

In [8]:
device = "cpu"

In [9]:
# Instatiate the physics config
robot_model = RobotModelConfig(robot_type="marv")
world_config = WorldConfig(
    x_grid=x_grid,
    y_grid=y_grid,
    z_grid=z_grid,
    grid_res=grid_res,
    max_coord=max_coord,
    k_stiffness=30000,
    suitable_mask=suit_mask,
)
physics_config = PhysicsEngineConfig(num_robots=num_robots)

Loading robot model from cache: /Users/davidkorcak/Documents/ctu/bachelors/flipper_training/.robot_cache/marv_0.010_192_bv0.01_whl0.02_trck0.05.pt
driving_direction: torch.Size([3])
body_bbox: torch.Size([6])
driving_part_bboxes: torch.Size([4, 6])
driving_part_masses: torch.Size([4])
joint_positions: torch.Size([4, 3])
joint_limits: torch.Size([2, 4])
joint_max_pivot_vels: torch.Size([4])
driving_part_movable_mask: torch.Size([4])
driving_part_points: torch.Size([4, 192, 3])
driving_part_inertias: torch.Size([4, 3, 3])
driving_part_cogs: torch.Size([4, 3])
body_points: torch.Size([256, 3])
body_inertia: torch.Size([3, 3])
body_cog: torch.Size([3])
thrust_directions: torch.Size([4, 192, 3])
joint_local_driving_part_pts: torch.Size([4, 192, 3])
joint_local_driving_part_cogs: torch.Size([4, 3])
RobotModelConfig for marv
Body mass: 47.8
Total mass: 67.8
Body bbox: tensor([ 0.3000,  0.1900,  0.4500, -0.3000, -0.1900, -0.0500])
Number of driving parts: 4
Driving part masses: tensor([5., 5.,

In [10]:
from flipper_training.observations import *
from functools import partial

obs = {
    "perception": partial(Heightmap, percep_shape=(128, 128), percep_extent=(1.0, 1.0, -1.0, -1.0)),
    "observation": partial(RobotStateVector),
}

obs["perception"]

functools.partial(<class 'flipper_training.observations.heightmap.Heightmap'>, percep_shape=(128, 128), percep_extent=(1.0, 1.0, -1.0, -1.0))

In [11]:
from flipper_training.rl_rewards.rewards import *

reward = RollPitchGoal(1000, -1000, 1.0, 1.0)

In [16]:
torch_env = Env(SimpleStabilizationObjective(),
                reward,
                obs,
                world_config,
                physics_config,
                robot_model,
                device,
                differentiable=False,
                batch_size=[num_robots])

In [17]:
torchrl.envs.utils.check_env_specs(torch_env)

2025-02-25 14:17:03,718 [torchrl][INFO] check_env_specs succeeded!


In [20]:
# Controls
speed = 0.5  # m/s forward
flipper_controls = torch.zeros(robot_model.num_driving_parts)
control_vec = (
    torch.cat([torch.full((4,), speed), flipper_controls]).unsqueeze(0).repeat(num_robots, 1)
)
control_vec.shape

torch.Size([1, 8])

In [21]:
control_td = TensorDict({"action": control_vec}, batch_size=num_robots)
control_td.shape

torch.Size([1])

In [22]:
torch_env.reset(reset_all=True)
torch_env.visualize_curr_state()

In [23]:
for i in range(100):
    o = torch_env.step(control_td)
    # control_td.pop("next")
torch_env.visualize_curr_state()